In [4]:
KNN_EXPERIMENT_FILE = "katib-knn-experiment.yaml"

In [5]:
import re

from IPython.utils.capture import CapturedIO


def get_resource(captured_io: CapturedIO) -> str:
    """
    Gets a resource name from `kubectl apply -f <configuration.yaml>`.

    :param str captured_io: Output captured by using `%%capture` cell magic
    :return: Name of the Kubernetes resource
    :rtype: str
    :raises Exception: if the resource could not be created
    """
    out = captured_io.stdout
    matches = re.search(r"^(.+)\s+created", out)
    if matches is not None:
        return matches.group(1)
    else:
        raise Exception(f"Cannot get resource as its creation failed: {out}. It may already exist.")

In [6]:
%%writefile $KNN_EXPERIMENT_FILE
apiVersion: kubeflow.org/v1alpha3
kind: Experiment
metadata:
  name: heart
  namespace: ekemini
spec:
  parallelTrialCount: 3
  maxTrialCount: 12
  maxFailedTrialCount: 3
  objective:
    type: maximize
    goal: 0.99
    objectiveMetricName: accuracy
  algorithm:
    algorithmName: bayesianoptimization
  metricsCollectorSpec:
    kind: StdOut
  parameters:
    - name: n_neighbors
      parameterType: int
      feasibleSpace:
        min: "5"
        max: "10"
    - name: weights
      parameterType: categorical
      feasibleSpace:
        list:
          - "uniform"
          - "distance"
    - name: algorithm
      parameterType: categorical
      feasibleSpace:
        list:
            - "auto"
            - "ball_tree"
            - "kd_tree"
            - "brute"
  trialTemplate:
    primaryContainerName: training-container
    trialParameters:
      - name: nneighbors
        description: Number of neighbors to use by default for kneighbors queries
        reference: n_neighbors
      - name: weights
        description: weight function used in prediction
        reference: weights
      - name: algorithm
        description: Algorithm used to compute the nearest neighbors
        reference: algorithm
    trialSpec:
      apiVersion: batch/v1
      kind: Job
      spec:
        template:
          metadata:
            annotations:
              sidecar.istio.io/inject: "false"
          spec:
            containers:
              - name: training-container
                image: "docker.io/mavencodevv/knnjobheart:v.0.1"
                command:
                  - python3
                  - -u
                  - /knnjobheart.py
                args:
                  - "--n_neighbors=${trialParameters.n_neighbors}"
                  - "--weights=${trialParameters.weights}"
                  - "--algorithm=${trialParameters.algorithm}"
            restartPolicy: Never

Overwriting katib-knn-experiment.yaml
